In [76]:
import pandas as pd, numpy as np, os, json
def open_json(file):
    with open(file, "r") as f:
        metr = json.load(f)
    return metr
def create_dfs(metr):
    l1,l2,l3,l4,l5,l6,l_pat, l_p, l_sens =[],[],[],[],[],[],[],[],[]
    for key,value in metr.items():
        a,b,c,d,e,f,k,p,sens= value[0]['dice'][0],value[0]['precision'][0], value[0]['recall'][0],value[0]['hd'][0],value[0]['Model'],value[0]['Number of Parameters'][0],key, value[0]['Dice Volumetric'],value[0]["Sensitivity Analysis"]
        l1.append(a),l2.append(b),l3.append(c),l4.append(d),l5.append(e),l6.append(f),l_pat.append(k),l_p.append(p),l_sens.append(sens)
    df = pd.DataFrame({"Dice":l1, "Precision":l2,"Recall":l3,"HD":l4,"Model":l5,"Parameters":l6, "Vol Dice": l_p},index=l_pat)
    return df,l_sens
def sens_best(fl, model, param):
    tmp = {}
    for item in range(len(fl[model][param])):
        for item1 in fl[model][param][item].keys():
            tmp.update({item1:[]})

    for item in range(len(fl[model][param])):
        for item1 in fl[model][param][item].keys():
            tmp[item1].append(fl[model][param][item][item1])
    nw = {}
    for key in tmp.keys():
        nw.update({key:np.mean(np.array(tmp[key]))})
    return nw

In [77]:
model_names = ['Unet', 'Attention_Unet', 'Vnet', 'USE_net', 'Dense2_Unet', 'Trans_Unet']
base_path = "EXPERIMENTS_V0.2"
fold = "fold_0"
weights_paths = ["metrics_final_Checkpoint_1024.json",
                 "metrics_final_Final_1024.json"]
params = ["Checkpoint_1024","Final_1024"]

dfs_mod = []
ls_sens_analysis = {}
for md_nm in model_names:
    wts_sens = {}
    for weight, param in zip(weights_paths, params):
        pth = [base_path, md_nm, fold, "Metrics_Json",weight]
        metr = open_json(os.path.join(*pth))
        df, list_sens = create_dfs(metr)
        wts_sens.update({param:list_sens})
        df["Size"] = param
        df["Fold"] = "Fold 0"
        dfs_mod.append(df)
    ls_sens_analysis.update({md_nm:wts_sens})

d2 = {}
for md_nm in model_names:
    d1 = {}
    for weight, param in zip(weights_paths, params):
        nw = sens_best(ls_sens_analysis, md_nm, param)
        d1.update({param:nw})
    d2.update({md_nm:d1})


In [87]:
d2
with open("EXPERIMENTS_V0.2/MaterialsGeneral/sens_analysis.json", "w") as file:
    json.dump(d2, file)

In [30]:
df_a = pd.concat(dfs_mod, axis = 0)

In [32]:
a = df_a.groupby(["Model","Size"]).median()

/tmp/ipykernel_1077896/3819943482.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.median is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  a = df_a.groupby(["Model","Size"]).median()


In [17]:
a

Dice  Precision    Recall            HD  \
Model          Size                                                           
Attention_Unet Cehckpoint_1024  0.229791   0.254474  0.165665  1.970406e+01   
               Final_1024       0.032256   0.062272  0.016405  2.072438e+01   
Dense2_Unet    Cehckpoint_1024  0.000000   0.000000  0.000000  3.470230e+01   
               Final_1024       0.000000   0.000000  0.000000  1.844674e+19   
Trans_Unet     Cehckpoint_1024  0.209665   0.281508  0.438057  2.240536e+01   
               Final_1024       0.223233   0.276884  0.340708  2.186321e+01   
USE_net        Cehckpoint_1024  0.187963   0.378151  0.131252  2.218671e+01   
               Final_1024       0.101657   0.395373  0.067873  2.289651e+01   
Unet           Cehckpoint_1024  0.188940   0.242709  0.216798  2.504496e+01   
               Final_1024       0.171566   0.209668  0.258435  2.518432e+01   
Vnet           Cehckpoint_1024  0.213721   0.495859  0.209747  2.191461e+01   
               Final_1024       0.103969   0.472904  0.057407  2.140093e+01   

                                 Parameters  
Model          Size                          
Attention_Unet Cehckpoint_1024   95418342.0  
               Final_1024        95418342.0  
Dense2_Unet    Cehckpoint_1024   27551681.0  
               Final_1024        27551681.0  
Trans_Unet     Cehckpoint_1024  344498881.0  
               Final_1024       344498881.0  
USE_net        Cehckpoint_1024  105620985.0  
               Final_1024       105620985.0  
Unet           Cehckpoint_1024  103768993.0  
               Final_1024       103768993.0  
Vnet           Cehckpoint_1024  105180865.0  
               Final_1024       105180865.0

In [35]:
a.to_csv("EXPERIMENTS_V0.2/MaterialsGeneral/results.csv")